# Clustering Crypto

In [32]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [33]:
# Use the following endpoint to fetch json data
#url = "https://min-api.cryptocompare.com/data/all/coinlist"
#response = requests.get(url).json()

In [34]:
# Create a DataFrame 
#crypto_df = pd.DataFrame(response['Data']).T
#crypto_df.head()

In [35]:
# Alternatively, use the provided csv file:
# The url pulled some weird data
file_path = Path("Resources/crypto_data.csv")

crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

### Data Preprocessing

In [36]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName', 'Algorithm', 'IsTrading' ,'ProofType' ,'TotalCoinsMined' ,'TotalCoinSupply']]
crypto_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

In [45]:
# Keep only cryptocurrencies that are trading
trading_cryptos_df = crypto_df[crypto_df['IsTrading'] == True]
trading_cryptos_df.head(10)

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
42      42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
365     365Coin       X11       True   PoW/PoS              NaN   
404     404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
611   SixEleven   SHA-256       True       PoW              NaN   
808         808   SHA-256       True   PoW/PoS     0.000000e+00   
1337  EliteCoin       X13       True   PoW/PoS     2.927942e+10   
2015  2015 coin       X11       True   PoW/PoS              NaN   
BTC     Bitcoin   SHA-256       True       PoW     1.792718e+07   
ETH    Ethereum    Ethash       True       PoW     1.076842e+08   
LTC    Litecoin    Scrypt       True       PoW     6.303924e+07   

     TotalCoinSupply  
42                42  
365       2300000000  
404        532000000  
611           611000  
808                0  
1337    314159265359  
2015               0  
BTC         21000000  
ETH                0  
LTC         84000000

In [38]:
# Keep only cryptocurrencies with a working algorithm
working_algo_crypto = trading_cryptos_df[trading_cryptos_df["Algorithm"] != "N/A"]

In [39]:
# Remove the "IsTrading" column
working_algo_crypto.drop(columns=["IsTrading"], axis=1, inplace=True)

In [40]:
# Remove rows with at least 1 null value
working_algo_crypto.dropna(inplace=True)

In [46]:
# Remove rows with cryptocurrencies having no coins mined
mined_cryptos = working_algo_crypto[working_algo_crypto['TotalCoinsMined'] > 0]

In [47]:
# Drop rows where there are 'N/A' text values
mined_cryptos = mined_cryptos[mined_cryptos != 'N/A']

In [53]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name = pd.DataFrame(mined_cryptos["CoinName"])
coin_name.head()

CoinName
42      42 Coin
404     404Coin
1337  EliteCoin
BTC     Bitcoin
ETH    Ethereum

In [54]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
mined_cryptos.drop(columns=['CoinName'], inplace=True)

In [55]:
# Create dummy variables for text features
text_dummies = pd.get_dummies(data=mined_cryptos, columns=["Algorithm", "ProofType"])
text_dummies.head()

TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
42       4.199995e+01              42                                 0   
404      1.055185e+09       532000000                                 0   
1337     2.927942e+10    314159265359                                 0   
BTC      1.792718e+07        21000000                                 0   
ETH      1.076842e+08               0                                 0   

      Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  Algorithm_Blake  \
42                0                  0                   0                0   
404               0                  0                   0                0   
1337              0                  0                   0                0   
BTC               0                  0                   0                0   
ETH               0                  0                   0                0   

      Algorithm_Blake2S  Algorithm_Blake2b  Algorithm_C11  ...  \
42                    0                  0              0  ...   
404                   0                  0              0  ...   
1337                  0                  0              0  ...   
BTC                   0                  0              0  ...   
ETH                   0                  0              0  ...   

      ProofType_PoW/PoS  ProofType_PoW/PoS   ProofType_PoW/PoW  \
42                    1                   0                  0   
404                   1                   0                  0   
1337                  1                   0                  0   
BTC                   0                   0                  0   
ETH                   0                   0                  0   

      ProofType_PoW/nPoS  ProofType_Pos  ProofType_Proof of Authority  \
42                     0              0                             0   
404                    0              0                             0   
1337                   0              0                             0   
BTC                    0              0                             0   
ETH                    0              0                             0   

      ProofType_Proof of Trust  ProofType_TPoS  \
42                           0               0   
404                          0               0   
1337                         0               0   
BTC                          0               0   
ETH                          0               0   

      ProofType_Zero-Knowledge Proof  ProofType_dPoW/PoW  
42                                 0                   0  
404                                0                   0  
1337                               0                   0  
BTC                                0                   0  
ETH                                0                   0  

[5 rows x 98 columns]

In [57]:
# Standardize data
text_dummies = StandardScaler().fit_transform(text_dummies)
text_dummies[0]

array([-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
       -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
       -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
       -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
       -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
       -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
       -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
       -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
       -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
       -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
       -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
       -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
       -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
       -0.0433963 , -0.39879994, -0.0433963 , -0.18168574, -0.04

### Reducing Dimensions Using PCA

In [60]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
pc = pca.fit_transform(text_dummies)

array([[-0.33651178,  1.01476888, -0.5993329 ],
       [-0.3198496 ,  1.01488153, -0.59974453],
       [ 2.31082382,  1.54374844, -0.65232013],
       ...,
       [ 0.3282572 , -2.32225683,  0.4600806 ],
       [-0.14404592, -2.15149814,  0.52883682],
       [-0.29181986,  0.84244762, -0.2796029 ]])

In [62]:
# Create a DataFrame with the principal components data
df = pd.DataFrame(pc, columns = ['pc1', 'pc2', 'pc3'])
df.head()

pc1       pc2       pc3
0 -0.336512  1.014769 -0.599333
1 -0.319850  1.014882 -0.599745
2  2.310824  1.543748 -0.652320
3 -0.149685 -1.314022  0.221921
4 -0.152125 -1.998267  0.432527

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [64]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for value in k:
    k_mean = KMeans(n_clusters=value)
    k_mean.fit(df)
    inertia.append(k_mean.inertia_)

# Create the Elbow Curve using hvPlot
elbow_curve_data = {"k": k, "inertia": inertia}
elbow_curve_df = pd.DataFrame(elbow_curve_data)
elbow_curve_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [73]:
# Initialize the K-Means model
k_model = KMeans(n_clusters=4, random_state=0)
# Fit the model
k_model.fit(df)
# Predict clusters
predictions = k_model.predict(df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
